In [1]:
from metrics.wallet import *

In [2]:
inconsistent_returns = {
    'ERR WRONGCERT', 'ERR UNSAT'
}

successful_returns = {'SAT', 'UNSAT'}

is_consistent_by_xp=(lambda x: not x['Checked answer'] in inconsistent_returns)
is_consistent_by_input=(lambda df: len(set(df['Checked answer'].unique()) & successful_returns) < 2)
is_success=(lambda x: x['Checked answer'] in successful_returns)

analysis = Analysis(
    input_file='config/metrics_scalpel.yml',
    is_consistent_by_xp=is_consistent_by_xp,
    is_consistent_by_input=is_consistent_by_input,
    is_success=is_success
)

/Users/wattez/Documents/phd/metrics/metrics/wallet/dataframe/builder.py:196: UserWarning: 4 experiments are inconsistent and are declared as unsuccessful.
  warn(f'{n_inconsistencies} experiments are inconsistent and are declared as unsuccessful.')


In [20]:
import re
family_re = re.compile(r'^XCSP\d\d/(.*?)/')
new_analysis = analysis.add_variable(
    new_var='family', 
    function=lambda x: family_re.match(x['input']).group(1),
    inplace=True
)

In [27]:
print(new_analysis.data_frame[['input', 'family']].sample(5).to_markdown())

|      | input                                                        | family        |
|-----:|:-------------------------------------------------------------|:--------------|
| 3641 | XCSP17/Primes/Primes-m1-p25/Primes-25-80-2-7.xml             | Primes        |
| 2992 | XCSP17/MaxCSP/MaxCSP-maxclique-s1/MaxCSP-brock-800-2.xml     | MaxCSP        |
| 2956 | XCSP17/MagicSquare/MagicSquare-sum-s1/MagicSquare-13-sum.xml | MagicSquare   |
| 7106 | XCSP18/GracefulGraph/GracefulGraph-K05-P02_c18.xml           | GracefulGraph |
| 4423 | XCSP17/QRandom/QRandom-mdd-7-25-5/mdd-7-25-5-56-09.xml       | QRandom       |


In [4]:
analysis.remove_variables(
    vars=['Category', 'Objective function'],
    inplace=True
)

In [5]:
analysis2 = analysis.add_variable(
    new_var='experiment_ware', 
    function=lambda x: x['experiment_ware'] + '-bis',
    inplace=False
)

In [6]:
analysis3 = analysis.add_analysis(
    analysis=analysis2,
    is_consistent_by_xp=is_consistent_by_xp,
    is_consistent_by_input=is_consistent_by_input,
    is_success=is_success
)

/Users/wattez/Documents/phd/metrics/metrics/wallet/dataframe/builder.py:196: UserWarning: 8 experiments are inconsistent and are declared as unsuccessful.
  warn(f'{n_inconsistencies} experiments are inconsistent and are declared as unsuccessful.')


In [7]:
analysis_vbs = analysis.add_virtual_experiment_ware(
    function=find_best_cpu_time_input, 
    xp_ware_set=None, 
    name='vbew'
)

In [8]:
len(analysis_vbs.data_frame)

8100

In [9]:
analysis.experiment_wares

array(['BTD 19.07.01', 'cosoco 2.O parallel', 'cosoco 2.0 parallel',
       'choco-solver 2019-09-16', 'choco-solver 2019-09-20',
       'choco-solver 2019-06-14', 'choco-solver 2019-09-20 parallel',
       'choco-solver 2019-09-24 parallel',
       'choco-solver 2019-06-14 parallel',
       'choco-solver 2019-09-16 parallel', 'AbsCon 2019-07-23',
       'Concrete 3.12.3', 'Fun-sCOP hybrid+CryptoMiniSat (2019-06-15)',
       'Fun-sCOP order+GlueMiniSat (2019-06-15)', 'cosoco 2.0',
       'cosoco 2', 'Fun-sCOP hybrid+ManyGlucose (2019-09-22)',
       'Fun-sCOP order+ManyGlucose (2019-09-22)',
       'Fun-sCOP hybrid+ManyGlucose (2019-06-15)',
       'Fun-sCOP order+ManyGlucose (2019-06-15)',
       'choco-solver 2019-09-24', 'Concrete 3.10', 'PicatSAT 2019-09-12',
       'Concrete 3.12.2'], dtype=object)

In [10]:
analysis_no_para = analysis.filter_analysis(lambda x: 'parallel' not in x['experiment_ware'])\
                            .filter_analysis(lambda x: 'Many' not in x['experiment_ware'])
analysis_no_para.experiment_wares

array(['BTD 19.07.01', 'choco-solver 2019-09-16',
       'choco-solver 2019-09-20', 'choco-solver 2019-06-14',
       'AbsCon 2019-07-23', 'Concrete 3.12.3',
       'Fun-sCOP hybrid+CryptoMiniSat (2019-06-15)',
       'Fun-sCOP order+GlueMiniSat (2019-06-15)', 'cosoco 2.0',
       'cosoco 2', 'choco-solver 2019-09-24', 'Concrete 3.10',
       'PicatSAT 2019-09-12', 'Concrete 3.12.2'], dtype=object)

In [11]:
analysis_clean = analysis_no_para.remove_experiment_wares({
    'Concrete 3.12.2', 
    'cosoco 2.0'
})

In [12]:
analysis_clean.experiment_wares

array(['BTD 19.07.01', 'choco-solver 2019-09-16',
       'choco-solver 2019-09-20', 'choco-solver 2019-06-14',
       'AbsCon 2019-07-23', 'Concrete 3.12.3',
       'Fun-sCOP hybrid+CryptoMiniSat (2019-06-15)',
       'Fun-sCOP order+GlueMiniSat (2019-06-15)', 'cosoco 2',
       'choco-solver 2019-09-24', 'Concrete 3.10', 'PicatSAT 2019-09-12'],
      dtype=object)

In [13]:
a = analysis.filter_inputs(
    function=lambda x: x['cpu_time'] > 100,
    how='all'
)

len(a.inputs)

59

In [14]:
print(len(analysis.delete_common_failed_inputs().inputs))
print(len(analysis.delete_common_solved_inputs().inputs))
print(len(analysis.keep_common_failed_inputs().inputs))
print(len(analysis.keep_common_solved_inputs().inputs))

273
244
27
56


In [15]:
analysis.groupby('family')

In [16]:
analysis_clean.error_table()

,input,experiment_ware,cpu_time,Checked answer,Wallclock time,Memory,Solver name,Solver version,timeout,error,success,missing,consistent_xp,consistent_input,family
7773,XCSP19/hcp/graph255.xml,cosoco 2,0.045418,ERR UNSAT,0.045142,0,cosoco,2,1200.0,True,False,False,False,True,hcp
7798,XCSP19/hcp/graph48.xml,choco-solver 2019-09-16,2306.850000,ERR WRONGCERT,583.697000,15530540,choco-solver,2019-09-16,1200.0,True,False,False,False,True,hcp


In [17]:
analysis_clean.description_table()

,analysis
n_experiment_wares,12
n_inputs,300
n_experiments,3600
n_missing_xp,0
n_inconsistent_xp,2
n_inconsistent_xp_due_to_input,0
more_info_about_variables,<analysis>.data_frame.describe(include='all')


In [18]:
analysis_clean.export('analysis.csv')
imported_analysis = import_analysis_from_file('analysis.csv')
imported_analysis.description_table()

,analysis
n_experiment_wares,12
n_inputs,300
n_experiments,3600
n_missing_xp,0
n_inconsistent_xp,2
n_inconsistent_xp_due_to_input,0
more_info_about_variables,<analysis>.data_frame.describe(include='all')
